## Notes 
<b> 2/18/2022 <b>
* For SAR, only projects that have empty phase completion dates are included. 
* Projects with NO allocation amount won't appear in SAR. 
* Wants to compare cols from previous SAR with current SAR b/c she has to bold any changes in black for CTC.
* For PAP, problem is prior col is all lumped together for projects before 2020? Highlands sheet does not differentiate between the years. Wants each year to be separated out.
    
<b> To do </b>
* Make sure all the values in date columns are replaced properly. 
* Change col names to match the new SAR report sent over 2/15.
* SAR: allocation amount col isn't from highlands sheet, it's taken by adding GGRF Alloc + PTA-SB1. Add them back in, add the column...

In [1]:
import numpy as np
import pandas as pd
import TIRCP_functions
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/tircp/"
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.2f}".format

In [2]:
#Load in Crosswalks 
FILE_NAME3 = "Allocation_PPNO_Crosswalk.csv"
allocation_ppno_crosswalk = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME3}")
    
#Allocation PPNO Crosswalk
FILE_NAME4 = "Projects_PPNO.xlsx"
project_ppno_crosswalk = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME4}")


### Loading in Scripts

In [ ]:
test_SAR = TIRCP_functions.semi_annual_report()

In [52]:
test_program = program_allocation_plan()

/opt/conda/lib/python3.9/site-packages/pandas/core/algorithms.py:1799: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  sorter = values.argsort()


In [3]:
alloc_test = TIRCP_functions.allocation()

(307, 49)

In [5]:
tableau_test = TIRCP_functions.tableau()

In [6]:
project_test = TIRCP_functions.project()

### Program Allocation Plan

In [51]:
def program_allocation_plan(): 
    ### LOAD IN SHEETS ### 
    df_project = project_test #REPLACE LATER
    df_allocation = alloc_test #REPLACE LATER
    #Only keeping certain columns
    df_project = (df_project[['Award_Year', 'Project_#','TIRCP_project_sheet','Local_Agency','Project_Title',
                              'PPNO', 'Unallocated_amt_project_sheet']]
                 )
    df_allocation = (df_allocation[['Award_Year','Award_Recipient', 'Implementing_Agency',
    'Components', 'PPNO','Phase',
    'Fiscal_Year_2020-2021', 'Fiscal_Year_2021-2022',
    'Fiscal_Year_2022-2023', 'Fiscal_Year_2023-2024',
    'Fiscal_Year_2024-2025', 'Fiscal_Year_2025-2026',
    'Fiscal_Year_2026-2027', 'Fiscal_Year_2027-2028',
    'Fiscal_Year_2028-2029', 'Fiscal_Year_2029-2030','CTC_Financial_Resolution',
    'Allocation_Date','Project_ID','SB1_Funding','GGRF_Funding','Allocation_Amt_Allocation_Sheet']]
                    ) 
    ### MERGE 2 SHEETS ###
    df_combined = df_allocation.merge(df_project, how = "left", on = ["PPNO", "Award_Year"])
    
    ### CLEAN UP ###
    #Fill in Project ID & CTC Fin Resolution with TBD so it'll show up
    df_combined[['Project_ID','CTC_Financial_Resolution']] = (df_combined[['Project_ID',
                                                            'CTC_Financial_Resolution']].fillna(value = 'TBD'))
    #Fill in missing dates with something random 
    missing_date = pd.to_datetime('2100-01-01')
    df_combined['Allocation_Date'] = df_combined['Allocation_Date'].fillna(missing_date)
    
    #Create Total_Amount Col
    df_combined['Total_Amount'] = df_combined['GGRF_Funding'] + df_combined['SB1_Funding']
    
    #Rename cols to the right names
    df_combined = (df_combined.rename(columns = {'TIRCP_project_sheet':'Award_Amount', 
                                                'Components': "Separable_Phases/Components",
                                                'CTC_Financial_Resolution': 'Allocation_Resolution',
                                                'SB1_Funding':'PTA-SB1_Amount', 
                                                'Unallocated_amt_project_sheet': 'Not_Allocated'})
                  ) 
    
    ### PIVOT ### 
    def pivot(df):
        df = df.groupby(['Award_Year','Project_#','Award_Amount','Not_Allocated','PPNO','Award_Recipient','Implementing_Agency',
        'Project_Title', 'Separable_Phases/Components','Phase','Project_ID','Allocation_Resolution','Allocation_Date']).agg({
        'Fiscal_Year_2020-2021': 'max',
        'Fiscal_Year_2021-2022': 'max', 'Fiscal_Year_2022-2023': 'max',
        'Fiscal_Year_2023-2024': 'max', 'Fiscal_Year_2024-2025': 'max',
        'Fiscal_Year_2025-2026': 'max', 'Fiscal_Year_2026-2027': 'max',
        'Fiscal_Year_2027-2028': 'max', 'Fiscal_Year_2028-2029': 'max',
        'Fiscal_Year_2029-2030': 'max', 'PTA-SB1_Amount': 'sum', 'GGRF_Funding':'sum',
         'Total_Amount':'sum'})
        return df 
    
    df_2015 = pivot(df_combined.loc[df_combined['Award_Year'] == 2015])
    df_2016 = pivot(df_combined.loc[df_combined['Award_Year'] == 2016])
    df_2018 = pivot(df_combined.loc[df_combined['Award_Year'] == 2018])
    df_2020 = pivot(df_combined.loc[df_combined['Award_Year'] == 2020])
    
    #GCS 
    with pd.ExcelWriter(f'{GCS_FILE_PATH}FUNCTION_TEST_PAP.xlsx') as writer:
        df_2015.to_excel(writer, sheet_name="2015_Cycle_1", index=True)
        df_2016.to_excel(writer, sheet_name="2016_Cycle_2", index=True)
        df_2018.to_excel(writer, sheet_name="2018_Cycle_3", index=True)
        df_2020.to_excel(writer, sheet_name="2020_Cycle_4", index=True)
        
    return df_combined
    

### Once I get the final data, compare the PPNO # across the 2 sheets 

In [ ]:
PPNO_project = set(project_test.PPNO.unique().tolist())
PPNO_allocation = set(alloc_test.PPNO.unique().tolist())

In [ ]:
PPNO_project - PPNO_allocation #checking for differences